In [1]:
from keras.applications import vgg16
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Flatten,BatchNormalization
from keras.activations import relu
from keras.initializers import glorot_uniform
from keras.regularizers import l2
# from keras.optimizers import Adam
from keras.models import Sequential
import os

In [16]:
from keras.metrics import Precision,Recall

In [2]:
data_gen = ImageDataGenerator()

In [3]:
base_dir = "F:/DATASET/hindi Devnagari handwritten Dataset"

In [4]:
train_base_dir = os.path.join(base_dir,"Train")
test_base_dir = os.path.join(base_dir,"Test")

In [5]:
training_datagen = data_gen.flow_from_directory(directory=train_base_dir,
                                                target_size=(32,32),color_mode="rgb"
                                                ,batch_size=50)

Found 78200 images belonging to 46 classes.


In [6]:
# cv_datagen = data_gen.flow_from_directory(directory="/content/drive/MyDrive/DevanagariHandwrittenCharacterDataset/cv",
#                                                 target_size=(32,32),color_mode="rgb",
#                                                 classes=os.listdir("/content/drive/MyDrive/DevanagariHandwrittenCharacterDataset/cv"),batch_size=15640)

In [7]:
testing_datagen = data_gen.flow_from_directory(directory=test_base_dir,
                                                target_size=(32,32),color_mode="rgb"
                                                ,batch_size=13800)

Found 13800 images belonging to 46 classes.


In [8]:
conv_base = vgg16.VGG16(include_top=False,input_shape=(32,32,3))

In [9]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0     

In [10]:
def unfreeze_layers(layer_name):

  trainable = False

  for layer in conv_base.layers:

    if layer.name == layer_name:
      trainable = True

    if trainable:
      layer.trainable = True
    else:
      layer.trainable = False

  return conv_base

In [18]:
def build_network():

  network = Sequential()
  conv_base = unfreeze_layers("block5_conv1")
  network.add(conv_base)
  network.add(Flatten())
  network.add(Dense(units=512,activation=relu,kernel_initializer=glorot_uniform))
  network.add(Dense(units=46,activation="softmax",kernel_initializer=glorot_uniform))

  network.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy" ,Precision(),Recall()])

  return network

In [19]:
network = build_network()

In [20]:
network.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 512)               262656    
                                                                 
 dense_3 (Dense)             (None, 46)                23598     
                                                                 
Total params: 15,000,942
Trainable params: 7,365,678
Non-trainable params: 7,635,264
_________________________________________________________________


In [ ]:
network.fit(training_datagen,validation_data=testing_datagen,epochs=2)

Epoch 1/2
1564/1564 [==============================] - ETA: 0s - loss: 3.8290 - accuracy: 0.0204 - precision: 0.0000e+00 - recall: 0.0000e+00